In [1]:
from google.cloud import bigquery
import polars as pl
import pandas as pd
import plotnine as p9
from pprint import pprint

In [2]:
# Set up a client
client = bigquery.Client(project = "umt-msba")

c:\Users\npleg\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 


In [3]:
# Write your query
query = """
    SELECT DISTINCT card_no
    FROM `umt-msba.transactions.transArchive_*`
    
"""
# Execute the query
query_job = client.query(query)

In [4]:
len(query_job)

TypeError: object of type 'QueryJob' has no len()

In [5]:
results = query_job.result()



In [6]:
results_df = results.to_dataframe()


c:\Users\npleg\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.


In [7]:
type(results_df)

pandas.core.frame.DataFrame

In [8]:
results_df.to_csv("test.txt", sep="\t", index = False)